In [1]:
from pathlib import Path

base = Path(r"C:\Users\Gresa Calliku\Documents\20250518_GC_RescU\RescU")

# Define folder structure
folders = [
    "data/geospatial",
    "data/satellite",
    "data/scripts",
    "models",
    "notebooks"
]

# Create folders
for folder in folders:
    (base / folder).mkdir(parents=True, exist_ok=True)

# Create ReadMe.md
(base / "ReadMe.md").touch()


In [2]:
from pathlib import Path

# Base path where structure should be created
base = Path(r"C:\Users\Gresa Calliku\Documents\20250518_GC_RescU\RescU")

# Folder structure
folders = [
    "data/geospatial",
    "data/satellite",
    "data/scripts",
    "models",
    "notebooks"
]

# Create folders and add ReadMe.md in each
for folder in folders:
    folder_path = base / folder
    folder_path.mkdir(parents=True, exist_ok=True)
    readme_file = folder_path / "ReadMe.md"
    readme_file.write_text(f"# {folder_path.name}\n\nThis folder contains {folder_path.name} files.\n")

# Create top-level ReadMe.md
top_readme = base / "ReadMe.md"
top_readme.write_text("# RescU Project\n\nThis repository contains geospatial data, models, and notebooks related to rescue simulation and accessibility analysis.")


137

In [4]:
!conda install -c conda-forge osmnx -y

Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: done

## Package Plan ##

  environment location: C:\Users\Gresa Calliku\miniconda3

  added / updated specs:
    - osmnx


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    affine-2.4.0               |     pyhd8ed1ab_1          19 KB  conda-forge
    blas-1.0                   |              mkl           1 KB  conda-forge
    blosc-1.21.3               |       h6c2663c_0          86 KB
    boost-cpp-1.82.0           |       h59b6b97_2          11 KB
    bottleneck-1.4.2           |  py313h2cb717b_0         145 KB
    branca-0.8.1               |     pyhd8ed1ab_0          29 KB  conda-forge
    brotli-1.0.9               |       hcfcfb64_8          19 KB  conda-forge
    brotli-bin-1.0.9           |       hcfcfb64_8          22 KB  conda-forge
    cairo-1.16.0               |       hc68a040_5         2.3 MB
    cfi

In [6]:
# this script can be called download_osm.py
import osmnx as ox

# Define your place of interest
place = "Berlin, Germany"

# Download building footprints
buildings = ox.geometries_from_place(place, tags={"building": True})

# Download road network
roads = ox.graph_from_place(place, network_type='drive')
roads_gdf = ox.graph_to_gdfs(roads, nodes=False)

# Download fire station locations
fire_stations = ox.geometries_from_place(place, tags={"emergency": "fire_station"})

# Save as GeoPackage for reuse
buildings.to_file("berlin_buildings.gpkg", layer='buildings', driver="GPKG")
roads_gdf.to_file("berlin_roads.gpkg", layer='roads', driver="GPKG")
fire_stations.to_file("berlin_fire_stations.gpkg", layer='fire_stations', driver="GPKG")

AttributeError: module 'osmnx' has no attribute 'geometries_from_place'

In [7]:
import osmnx as ox
print(ox.__version__)

2.0.3


In [8]:
import osmnx as ox

# Enable logging (optional)
ox.config(log_console=True)

# Define your area of interest
place = "Berlin, Germany"

# Define tags (e.g., all buildings)
tags = {"building": True}

# Get building footprints as GeoDataFrame
buildings = ox.features_from_place(place, tags=tags)

# Get drivable street network as graph
roads = ox.graph_from_place(place, network_type="drive")


AttributeError: module 'osmnx' has no attribute 'config'

In [9]:
import osmnx as ox
import logging

# Optional: enable logging
logging.basicConfig(level=logging.INFO)

# Define area of interest
place = "Berlin, Germany"
tags = {"building": True}

# Download building footprints
buildings = ox.features_from_place(place, tags=tags)

# Download road network
roads = ox.graph_from_place(place, network_type="drive")


MemoryError: Unable to allocate 5.91 GiB for an array with shape (1320, 600937) and data type object

In [ ]:
# download_osm.py

import osmnx as ox

# Define your place of interest
place = "Berlin, Germany"

# Download building footprints
buildings = ox.features_from_place(place, tags={"building": True})

# Download road network and convert to GeoDataFrame
roads_graph = ox.graph_from_place(place, network_type='drive')
roads_gdf = ox.graph_to_gdfs(roads_graph, nodes=False)

# Download fire station locations
fire_stations = ox.features_from_place(place, tags={"emergency": "fire_station"})

# Save to GeoPackages
buildings.to_file("berlin_buildings.gpkg", layer='buildings', driver="GPKG")
roads_gdf.to_file("berlin_roads.gpkg", layer='roads', driver="GPKG")
fire_stations.to_file("berlin_fire_stations.gpkg", layer='fire_stations', driver="GPKG")


In [10]:
%run download_osm.py

Exception: File `'download_osm.py'` not found.

In [ ]:
# download_sentinel.py

from sentinelsat import SentinelAPI, geojson_to_wkt
from datetime import date
import osmnx as ox

# Sentinel credentials (replace with your own Copernicus credentials)
username = 'your_username'
password = 'your_password'
api_url = 'https://scihub.copernicus.eu/dhus'

# Connect to API
api = SentinelAPI(username, password, api_url)

# Get bounding geometry for Berlin
area_gdf = ox.geocode_to_gdf("Berlin, Germany")
area_geom = area_gdf.geometry.values[0]
area_wkt = area_geom.to_wkt()  # Convert to WKT for the API

# Query Sentinel-2 products
products = api.query(area_wkt,
                     date=('20240101', '20240115'),
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 10))

# Check and download the first available product
if products:
    for product_id, product_info in products.items():
        print(f"Downloading: {product_info['title']}")
        api.download(product_id)
        break
else:
    print("No products found for the specified query.")


In [11]:
import osmnx as ox

# Smaller area to avoid memory errors
place = "Mitte, Berlin, Germany"
tags = {"building": True}

# Download building footprints
buildings = ox.features_from_place(place, tags=tags)

# Download drivable street network
roads_graph = ox.graph_from_place(place, network_type='drive')
roads_gdf = ox.graph_to_gdfs(roads_graph, nodes=False)

# Download fire stations
fire_stations = ox.features_from_place(place, tags={"emergency": "fire_station"})

# Save to GeoPackage (within a local folder structure if desired)
buildings.to_file("mitte_buildings.gpkg", layer="buildings", driver="GPKG")
roads_gdf.to_file("mitte_roads.gpkg", layer="roads", driver="GPKG")
fire_stations.to_file("mitte_fire_stations.gpkg", layer="fire_stations", driver="GPKG")



InsufficientResponseError: No matching features. Check query location, tags, and log.